# Donut Banner Encoding

**Spoilers**: This notebook demonstrates the encoding of the scrolling "IOCCC 2006" banner. If you want to figure it out, ignore this.



## Process

* Run-length encoding
* Prefix code trees (similar to Huffman encoding) on both the charactors and the run-lengths
* The resulting binary string is broken up into 6 bit blocks, read least significant bit to most
* The blocks are converted to ascii, taking the 64 chars from '>' to '~'

## Prefix code trees

The prefix code trees are represented in two strings each, corresponding to 0 and 1 in the binary input. The tree is traversed by pulling the next index from either the 0 or 1 array until a negative number is reached. This will probably take some pictures to understand.

In [1]:
# Using raw literal because the \\ are not escapes.
banner = r'''
.___________~~_________ _________ _________~~~_______________~~_______~~~~________
|   \_____  \~\_   ___ \\_   ___ \\_   ___ \~~\_____  \   _  \~\   _  \~~/  _____/
|   |/   |   \/    \  \//    \  \//    \  \/~~~/  ____/  /_\  \/  /_\  \/   __  \
|   /    |    \     \___\     \___\     \____~/       \  \_/   \  \_/   \  |__\  \
|___\_______  /\______  /\______  /\______  /~\_______ \_____  /\_____  /\_____  /
~~~~~~~~~~~~\/~~~~~~~~\/~~~~~~~~\/~~~~~~~~\/~~~~~~~~~~\/~~~~~\/~~~~~~~\/~~~~~~~\/
'''.strip() + "\n"


In [2]:
def run_length_encode(text):
    prev_ch = None
    count = 0

    rle_entries = []

    for ch in text:
        if ch == prev_ch:
            count += 1
        else:
            if prev_ch:
                rle_entries.append((prev_ch, count))
            prev_ch = ch
            count = 1
    if prev_ch:
        rle_entries.append((prev_ch, count))
    return rle_entries

In [3]:
run_length_encode(banner)[:5]

[('.', 1), ('_', 11), ('~', 2), ('_', 9), (' ', 1)]

In [4]:
# There is more work left to show here about how these strings become these dicts.


# 0: "bcd\\a[g" => [-1, 0, 1, -7, -2, -8, 4]
# 1: "^`_e]fh" => [-5, -3, -4, 2, -6, 3, 5]

char_lookup = {-8: '10',
 -7: '110',
 -6: '01',
 -5: '111001',
 -4: '1111',
 -3: '11101',
 -2: '00',
 -1: '111000'}


# 0: "_ac[]\\YZijkm" =>  [-4, -2, 0, -8, -6, -7, -10, -9, 6, 7, 8, 10]
# 1: "`bd^efghXWlV" => [-3, -1, 1, -5, 2, 3, 4, 5, -11, -12, 9, -13]
count_lookup = {-13: '1',
 -12: '011',
 -11: '001',
 -10: '0000',
 -9: '0100',
 -8: '010110',
 -7: '01010',
 -6: '00010',
 -5: '010111',
 -4: '0001100',
 -3: '0001101',
 -2: '0001110',
 -1: '0001111'}

def invert_dict(d):
    return {v: k for k, v in d.items()}

bin_to_count_code = invert_dict(count_lookup)

bin_to_char_code = invert_dict(char_lookup)

char_lookup_string = " /\\\n~|_."

def count_to_binary_code(count):
    if (count > 13):
        count = 13
    count_offset = 14
    return count_lookup[count - count_offset]

def char_to_binary_code(ch):
    if ch not in char_lookup_string:
        print(f'{ch} is not one of "{char_lookup_string}"')
    idx = char_lookup_string.index(ch)
    char_offset = 8
    return char_lookup[idx - char_offset]

def count_code_to_count(code):
    count_offset = 14
    count = code + count_offset
    if (count == 13):
        count = 15
    
    return count

def char_code_to_char(code):
    char_offset = 8
    return char_lookup_string[code + char_offset]


In [5]:
count_to_binary_code(3)

'001'

In [6]:
char_to_binary_code("\n")

'111001'

'111000100000110111110110001011110100010111101000101111111001000001111111101100010101111000000000101110011111011100010110001001001101111111011001100010000110101011001100010000110101011001100010000110101111110110110001001001101110001001100110111111101110001001100110111111011110110011000100110111100111110111000111101111011000111101110001011110110000001110011011110011100000011100110111100111000000111001101111011111001110110011000000110110011110100101110011011110110011110100101110011011110110001000111001101111100111110111000111011000001110111000000111001000110000101110010001100001011100100011000000111111101100101001110011011001110110001011100110110011101100010111001111101100011011100110111110011111011000010110001010100111101011000101101001111010110001011010011110101100010110100111101111110110001010101011000100100111101011000100100111101011000100100111101111001111110001110011110111110001001111011111000100111101111100010011110111110001100011110111110100011110111110101001111011111010100111101

In [9]:
magic_string_dense = "E?yYrIxC{e^}KhE>[|LXbj}dOVsJ@idOV{Yab[bW}[bW}\\qFywyv{Dma\\AZtq?Lyw>e{|Zq>Y\\gq\\qI[tYBe{wyvDZEvBA[`_Lo>}KcqdYrWqKxzKtW]|DXRwsfcUaT\\KXw{YRsFwsFwsFw{zaqyaz|FmMpyaoyI\\]cuUw{J"

magic_string_nums = [ord(ch) - 62 for ch in magic_string_dense]
print(magic_string_nums[:5])


magic_string_binary_segments = [bin(num)[2:] for num in magic_string_nums]
print(magic_string_binary_segments[:5])

# Zero pad to six-bits and reverse because of the order they are read in
magic_string_rev_binary_segments = [bstr.zfill(6)[::-1] for bstr in magic_string_binary_segments]
print(magic_string_rev_binary_segments[:5])

magic_string = ''.join(magic_string_rev_binary_segments)
print(magic_string[:20])

[7, 1, 59, 27, 52]
['111', '1', '111011', '11011', '110100']
['111000', '100000', '110111', '110110', '001011']
11100010000011011111


In [10]:
def read_bin_string_to_char_count_code_pairs(magic_string):
    char_count_pairs = []
    ms_idx = 0
    cur_bin_str = ''
    while ms_idx < len(magic_string):
        pair_start_idx = ms_idx
        while cur_bin_str not in bin_to_char_code:
            if ms_idx >= len(magic_string):
                print(f"Overflow at ms_idx {ms_idx} pair_start_idx {pair_start_idx}")
            cur_bin_str += magic_string[ms_idx]
            ms_idx += 1
        char_code = bin_to_char_code[cur_bin_str]
        cur_bin_str = ''
        while cur_bin_str not in bin_to_count_code:
            if ms_idx >= len(magic_string):
                print(f"Overflow at ms_idx {ms_idx} pair_start_idx {pair_start_idx}")
            cur_bin_str += magic_string[ms_idx]
            ms_idx += 1
        count_code = bin_to_count_code[cur_bin_str]
        cur_bin_str = ''
        char_count_pairs.append((char_code, count_code))
    return char_count_pairs
    

In [11]:
char_count_code_pairs = read_bin_string_to_char_count_code_pairs(magic_string[:1006])

print(char_count_code_pairs[:5])

char_count_pairs = [(char_code_to_char(char_code), count_code_to_count(count_code)) for (char_code, count_code) in char_count_code_pairs]

print(char_count_pairs)

expanded_chars = [char * count for (char, count) in char_count_pairs]
print(expanded_chars[:5])
rendered_banner = ''.join(expanded_chars)

print(rendered_banner)

[(-1, -13), (-2, -3), (-4, -12), (-2, -5), (-8, -13)]
[('.', 1), ('_', 11), ('~', 2), ('_', 9), (' ', 1), ('_', 9), (' ', 1), ('_', 9), ('~', 3), ('_', 15), ('~', 2), ('_', 7), ('~', 4), ('_', 8), ('\n', 1), ('|', 1), (' ', 3), ('\\', 1), ('_', 5), (' ', 2), ('\\', 1), ('~', 1), ('\\', 1), ('_', 1), (' ', 3), ('_', 3), (' ', 1), ('\\', 2), ('_', 1), (' ', 3), ('_', 3), (' ', 1), ('\\', 2), ('_', 1), (' ', 3), ('_', 3), (' ', 1), ('\\', 1), ('~', 2), ('\\', 1), ('_', 5), (' ', 2), ('\\', 1), (' ', 3), ('_', 1), (' ', 2), ('\\', 1), ('~', 1), ('\\', 1), (' ', 3), ('_', 1), (' ', 2), ('\\', 1), ('~', 2), ('/', 1), (' ', 2), ('_', 5), ('/', 1), ('\n', 1), ('|', 1), (' ', 3), ('|', 1), ('/', 1), (' ', 3), ('|', 1), (' ', 3), ('\\', 1), ('/', 1), (' ', 4), ('\\', 1), (' ', 2), ('\\', 1), ('/', 2), (' ', 4), ('\\', 1), (' ', 2), ('\\', 1), ('/', 2), (' ', 4), ('\\', 1), (' ', 2), ('\\', 1), ('/', 1), ('~', 3), ('/', 1), (' ', 2), ('_', 4), ('/', 1), (' ', 2), ('/', 1), ('_', 1), ('\\', 1), ('

In [12]:
assert rendered_banner[:20] == banner[:20]
assert rendered_banner[:30] == banner[:30]
assert rendered_banner[:50] == banner[:50]
assert rendered_banner[-1] == banner[-1] # Newline
# print(list(rendered_banner[-20:]))
# print(list(banner[-20:]))
assert rendered_banner == banner

In [13]:
rle = run_length_encode(banner)
combined_binary_codes = [ f"{char_to_binary_code(ch)}{count_to_binary_code(count)}" for (ch, count) in rle]

magic_string_from_banner = ''.join(combined_binary_codes)
magic_string_from_banner

'111000100000110111110110001011110100010111101000101111111001000001111111101100010101111000000000101110011111011100010110001001001101111111011001100010000110101011001100010000110101011001100010000110101111110110110001001001101110001001100110111111101110001001100110111111011110110011000100110111100111110111000111101111011000111101110001011110110000001110011011110011100000011100110111100111000000111001101111011111001110110011000000110110011110100101110011011110110011110100101110011011110110001000111001101111100111110111000111011000001110111000000111001000110000101110010001100001011100100011000000111111101100101001110011011001110110001011100110110011101100010111001111101100011011100110111110011111011000010110001010100111101011000101101001111010110001011010011110101100010110100111101111110110001010101011000100100111101011000100100111101011000100100111101111001111110001110011110111110001001111011111000100111101111100010011110111110001100011110111110100011110111110101001111011111010100111101

In [14]:
assert magic_string_from_banner == magic_string[:1006]

In [15]:
def chunk_string(string, chunk_size):
    return [ string[i:i+chunk_size] for i in range(0, len(string), chunk_size)]

magic_string_from_banner_chunks = chunk_string(magic_string_from_banner, 6)
# Last chunk not 6 long, not padding seems to work.

print(magic_string_from_banner_chunks[:5])
magic_string_from_banner_reverse_chunks = [bstr[::-1] for bstr in magic_string_from_banner_chunks]
print(magic_string_from_banner_reverse_chunks[:5])

magic_string_from_banner_nums = [int(bstr, 2) for bstr in magic_string_from_banner_reverse_chunks]
print(magic_string_from_banner_nums[:5])
magic_string_from_banner_chars = [chr(num + 62) for num in magic_string_from_banner_nums]
print(magic_string_from_banner_chars[:5])

magic_string_from_banner_dense = ''.join(magic_string_from_banner_chars)

print(magic_string_from_banner_dense[:20])

['111000', '100000', '110111', '110110', '001011']
['000111', '000001', '111011', '011011', '110100']
[7, 1, 59, 27, 52]
['E', '?', 'y', 'Y', 'r']
E?yYrIxC{e^}KhE>[|LX


In [153]:
# print(magic_string_from_banner_dense[-5:])
# print(magic_string_dense[-5:])
assert magic_string_from_banner_dense == magic_string_dense

In [16]:
new_banner = r'''
__________~~~~~~~~~~~_____~~~~~~~~~~~~~~~~~~~__
\______   \~~____~~_/ ____\_____~~~~~____~~_/  |_~~~____~~_______
~|       _/_/ __ \~\   __\~\__  \~~_/ ___\~\   __\ /  _ \~\_  __ \
~|    |   \\  ___/~~|  |~~~~/ __ \_\  \___~~|  |~~|  |_| |~|  |~\/
~|____|_  /~\___ |~~|__|~~~|____  /~\___ |~~|__|~~~\____/~~|__|
~~~~~~~~\/~~~~~~\/~~~~~~~~~~~~~~\/~~~~~~\/
'''.strip() + "\n"

In [17]:
def banner_to_magic_string(banner):
    rle = run_length_encode(banner)
    combined_binary_codes = [ f"{char_to_binary_code(ch)}{count_to_binary_code(count)}" for (ch, count) in rle]

    return ''.join(combined_binary_codes)
new_banner_magic_string = banner_to_magic_string(new_banner)
print(new_banner_magic_string)
print(len(new_banner_magic_string))

000001100111100011010001001111000111100011111001101100010110100010111111011000000111101100111011010000000110001001111010000000011110110011101100111110110011111001000000111101100010101110011111111110111001010001110100111011010001110101111111011100010001101111111011000111001101111110110011101101000010111111101110001000110111011101100110011010111111101100110011000111010111110011111111110111000001110111000101011100110000111011111011111011100111110111111000011011010001110101100101110011011000011111011111011100111110111111011111011100111110110011110111011110111111111101110011111011111110111101111001111111111011000000111011001100111101111110110000110111101111110111110110001111101111110011110110000001001111011111101100001101111011111101111101100011111011111100101100000011011111011111011000111110111110011111100010011110111110101100111101111100011110111101111101011001111011110011
882
